# Braindecode

Attempting to classify data using (convolutional) neural nets to compare performance.

# TODO: Refactor data loading in Main.ipynb and call from here to get epochs

In [ ]:
%run ./Main.ipynb

In [ ]:
# To use braindecode we need to transform our data to the braindecode format

from braindecode.datasets import create_from_X_y

# This wants X to be in the shape (x_trials, n_channels, n_samples)
Xb, yb, subjb, imgb, sessb, tsb = zip(*epochs)

Xb = [x.to_numpy().T for x in Xb]
print(len(Xb), Xb[0].shape)

yb = np.array([0 if yy == 'code' else 1 for yy in y])
print(yb.shape)

windows_dataset = create_from_X_y(
    Xb, yb, drop_last_window=True, sfreq=sfreq, ch_names=list(eeg.columns),
    window_stride_samples=512,
    window_size_samples=1024,
)

In [ ]:
windows_dataset.description['group'] = subjb
windows_dataset.description

In [ ]:
# Homegrown LORO
splitted = windows_dataset.split('group')

# Subject to use for validation
subj_val = 5

train_sets = [v for k, v in splitted.items() if k != str(subj_val)]

train_set = train_sets[0]
for ts in train_sets[1:]:
    train_set += ts
    
valid_set = splitted[str(subj_val)]

In [ ]:
print(train_set[0])
print(train_set[0][0].shape)

In [ ]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, Deep4Net

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
if cuda:
    print("CUDA available!")
    torch.backends.cudnn.benchmark = True
    
# Set random seed to be able to reproduce results
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

# Extract number of chans and time steps from dataset
n_classes = len(set(y))
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]

print(f"classes:   {n_classes}")
print(f"channels:  {n_chans}")
print(f"samples per window:  {input_window_samples}")

models = [
    (
        ShallowFBCSPNet(
            n_chans,
            n_classes,
            input_window_samples=input_window_samples,
            final_conv_length='auto',
        ), 
        {"lr": 0.0625 * 0.01, "weight_decay": 0}
    ),
    (
        Deep4Net(
            n_chans, 
            n_classes, 
            input_window_samples=input_window_samples,
            final_conv_length='auto'
        ), 
        {"lr": 1 * 0.01, "weight_decay": 0.5 * 0.001}
    )
]

In [ ]:
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGClassifier

batch_size = 64
n_epochs = 10

nn_clfs = []
for model, params in models:
    clf = EEGClassifier(
        model,
        criterion=torch.nn.NLLLoss,
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(valid_set),  # using valid_set for validation
        optimizer__lr=params["lr"],
        optimizer__weight_decay=params["weight_decay"],
        batch_size=batch_size,
        callbacks=[
            "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
        ],
        device='cuda' if cuda else 'cpu',
    )
    nn_clfs.append(clf)

In [ ]:
# Model training for a specified number of epochs. `y` is None as it is already supplied in the dataset.
for clf in nn_clfs:
    logger.info(f"==== Training {clf.module.__class__.__name__} ====")
    
    # Send model to GPU
    if cuda:
        clf.module.cuda()
        
    # FIXME: Remove try/except when error is resolved
    try:
        clf.fit(train_set, y=None, epochs=n_epochs)
    except Exception as e:
        logger.exception(e)